# EuroSAT Project - Phase 2 & 3
Model Building, Training, Evaluation & Saving

In [ ]:
# EuroSAT Land Use Classification - Phase 2 & 3: Model Building, Training & Evaluation

In [ ]:
## Step 1: Load Processed Data

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image

In [7]:
import os
import numpy as np
from PIL import Image

def load_data(data_dir, img_size=(64, 64)):
    images, labels = [], []
    label_map = {}

    for idx, cat in enumerate(sorted(os.listdir(data_dir))):
        label_map[idx] = cat
        for file in os.listdir(os.path.join(data_dir, cat)):
            img_path = os.path.join(data_dir, cat, file)
            img = Image.open(img_path).resize(img_size)
            images.append(np.array(img))
            labels.append(idx)

    return np.array(images), np.array(labels), label_map

In [9]:
data_dir = "EuroSAT/2750"
X, y, label_map = load_data(data_dir)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Label map:", label_map)

X shape: (27000, 64, 64, 3)
y shape: (27000,)
Label map: {0: 'AnnualCrop', 1: 'Forest', 2: 'HerbaceousVegetation', 3: 'Highway', 4: 'Industrial', 5: 'Pasture', 6: 'PermanentCrop', 7: 'Residential', 8: 'River', 9: 'SeaLake'}


In [13]:
data_path = "EuroSAT/2750"
X, y, label_map = load_data(data_path)
X = X / 255.0  # Normalize

In [15]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
## Step 2: Build CNN Model with TensorFlow/Keras

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [21]:
num_classes = len(np.unique(y_train))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Input(shape=(64, 64, 3)),  
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,626,442 (6.20 MB)

 Trainable params: 1,626,442 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
## Step 3: Train the Model

In [31]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 143s 256ms/step - accuracy: 0.3372 - loss: 1.7400 - val_accuracy: 0.5437 - val_loss: 1.2503
Epoch 2/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 133s 247ms/step - accuracy: 0.5788 - loss: 1.1698 - val_accuracy: 0.7037 - val_loss: 0.8176
Epoch 3/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 137s 254ms/step - accuracy: 0.6602 - loss: 0.9608 - val_accuracy: 0.7509 - val_loss: 0.7231
Epoch 4/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 129s 239ms/step - accuracy: 0.7072 - loss: 0.8398 - val_accuracy: 0.7495 - val_loss: 0.6770
Epoch 5/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 145s 244ms/step - accuracy: 0.7279 - loss: 0.7672 - val_accuracy: 0.7681 - val_loss: 0.6825
Epoch 6/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 132s 244ms/step - accuracy: 0.7359 - loss: 0.7435 - val_accuracy: 0.7625 - val_loss: 0.6696
Epoch 7/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 133s 246ms/step - accuracy: 0.7584 - loss: 0.6902 - val_accuracy: 0.7863 - val_loss: 0.6276
Epoch 8/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 130s 241ms/step - accuracy: 0.7679 -

In [ ]:
## Step 4: Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
## Step 5: Plot Training History

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12, 5))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
## Step 6: Save the Model

In [ ]:
model.save("eurosat_cnn_model.h5")
print("Model saved as eurosat_cnn_model.h5")